In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'co2': 0.0008}
band = [5]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.7, 1.6, 1.8), (1.8, 1.7), (1.5, 1.6, 1.7, 1.8)]
  wgt = [(0.6, 0.4, 0.7), (0.7, 0.4), (0.3, 0.4, 0.85, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.232615                    1  0.006285
0.000750        2  0.261483                    2  0.044187
0.001052        3  0.381570                    3  0.138886
0.001476        4  0.512103                    4  0.246371
0.002070        5  0.648078                    5  0.365559
0.002904        6  0.785376                    6  0.495072
0.004074        7  0.917646                    7  0.636419
0.005714        8  1.037317                    8  0.786908
0.008015        9  1.135983                    9  0.938169
0.011243       10  1.205070                   10  1.081197
0.015771       11  1.238039                   11  1.204167
0.022122       12  1.233184                   12  1.292413
0.031031       13  1.196626                   13  1.329054
0.043528       14  1.105669                   14  1.268438
0.061057       15  0.951378                   15  1.086736
0.085645       16  0.793903                   16  0.850605
0.120136       17  0.672224                   17  0.640660
0.168516       18  0.586777                   18  0.503845
0.236378       19  0.538000                   19  0.453635
0.331549       20  0.529992                   20  0.471967
0.465100       21  0.582266                   21  0.556026
0.652400       22  0.541944                   22  0.542911
0.915100       23  0.396636                   23  0.411750
1.283650       24  0.289850                   24  0.302845
1.800600       25  0.221554                   25  0.229830
2.525700       26  0.169289                   26  0.177028
3.542800       27  0.125324                   27  0.136961
4.969550       28  0.095582                   28  0.108515
6.970850       29  0.081653                   29  0.092863
9.778100       30  0.071078                   30  0.077498
13.715850      31  0.058545                   31  0.059373
19.239350      32  0.044952                   32  0.039178
26.987250      33  0.039129                   33  0.030038
37.855300      34  0.041261                   34  0.035068
53.100050      35  0.042480                   35  0.041953
73.887500      36  0.043170                   36  0.046781
97.662500      37  0.043241                   37  0.047526
121.437500     38  0.039939                   38  0.043159
145.212500     39  0.034900                   39  0.036592
168.987500     40  0.030487                   40  0.030845
192.762500     41  0.026014                   41  0.025319
216.537500     42  0.021220                   42  0.019682
240.312500     43  0.015787                   43  0.013280
264.087500     44  0.008907                   44  0.005230
287.862500     45 -0.000593                   45 -0.005382
311.637500     46 -0.004130                   46 -0.009349
335.412500     47  0.003832                   47 -0.000929
359.187500     48  0.014274                   48  0.010464
382.962500     49  0.022526                   49  0.019515
406.737500     50  0.029815                   50  0.027366
430.512500     51  0.036549                   51  0.034604
454.287500     52  0.042874                   52  0.041447
478.062500     53  0.048863                   53  0.048213
501.837500     54  0.054540                   54  0.054678
525.612500     55  0.059900                   55  0.060767
549.387500     56  0.064885                   56  0.066620
573.162500     57  0.069414                   57  0.072080
596.937500     58  0.073377                   58  0.076975
620.712500     59  0.076662                   59  0.081018
644.487500     60  0.079203                   60  0.084303
668.262500     61  0.081001                   61  0.086581
692.037500     62  0.082176                   62  0.088151
715.812500     63  0.082909                   63  0.089186
739.587500     64  0.083463                   64  0.089949
763.362500     65  0.084143                   65  0.090740
78

# Fluxes by Layer

CRD                      CLIRAD             \
                        flug      fldg      fnetg      i       flug   
pressure    level                                                     
0.000000    1     -15.670442  0.000000 -15.670442     11 -15.495968   
0.000624    2     -15.670496  0.000071 -15.670425     11 -15.496018   
0.000876    3     -15.670517  0.000100 -15.670417     11 -15.496038   
0.001229    4     -15.670547  0.000145 -15.670401     11 -15.496066   
0.001723    5     -15.670586  0.000215 -15.670371     11 -15.496103   
0.002417    6     -15.670640  0.000322 -15.670318     11 -15.496153   
0.003391    7     -15.670710  0.000483 -15.670227     11 -15.496220   
0.004757    8     -15.670802  0.000723 -15.670079     11 -15.496308   
0.006672    9     -15.670921  0.001077 -15.669844     11 -15.496423   
0.009359    10    -15.671071  0.001589 -15.669482     11 -15.496569   
0.013128    11    -15.671257  0.002313 -15.668944     11 -15.496750   
0.018415    12    -15.671479  0.003310 -15.668169     11 -15.496967   
0.025830    13    -15.671734  0.004649 -15.667085     11 -15.497214   
0.036232    14    -15.672002  0.006391 -15.665611     11 -15.497472   
0.050823    15    -15.672257  0.008557 -15.663700     11 -15.497717   
0.071291    16    -15.672502  0.011109 -15.661393     11 -15.497947   
0.100000    17    -15.672732  0.014039 -15.658693     11 -15.498155   
0.140271    18    -15.672902  0.017416 -15.655487     11 -15.498291   
0.196760    19    -15.672927  0.021367 -15.651560     11 -15.498272   
0.275997    20    -15.672643  0.026133 -15.646510     11 -15.497943   
0.387100    21    -15.671677  0.032142 -15.639535     11 -15.496961   
0.543100    22    -15.668792  0.040016 -15.628775     11 -15.494056   
0.761700    23    -15.663555  0.048813 -15.614742     11 -15.488748   
1.068500    24    -15.657439  0.057112 -15.600327     11 -15.482470   
1.498800    25    -15.651128  0.065575 -15.585553     11 -15.475888   
2.102400    26    -15.644644  0.074933 -15.569711     11 -15.469003   
2.949000    27    -15.638140  0.085406 -15.552734     11 -15.461973   
4.136600    28    -15.632278  0.097174 -15.535104     11 -15.455404   
5.802500    29    -15.627592  0.111350 -15.516242     11 -15.449807   
8.139200    30    -15.623656  0.130016 -15.493640     11 -15.444667   
11.417000   31    -15.620410  0.154367 -15.466042     11 -15.439895   
16.014700   32    -15.619122  0.184965 -15.434157     11 -15.436892   
22.464000   33    -15.623279  0.223464 -15.399815     11 -15.439766   
31.510500   34    -15.635206  0.277323 -15.357883     11 -15.451170   
44.200100   35    -15.653842  0.357982 -15.295860     11 -15.469295   
62.000000   36    -15.680498  0.474208 -15.206290     11 -15.495704   
85.775000   37    -15.715786  0.631077 -15.084709     11 -15.531785   
109.550000  38    -15.750580  0.787652 -14.962928     11 -15.568195   
133.325000  39    -15.787757  0.937311 -14.850447     11 -15.607597   
157.100000  40    -15.830846  1.078690 -14.752156     11 -15.653253   
180.875000  41    -15.880579  1.214283 -14.666295     11 -15.705832   
204.650000  42    -15.937823  1.344791 -14.593032     11 -15.766312   
228.425000  43    -16.003857  1.470588 -14.533269     11 -15.836842   
252.200000  44    -16.080733  1.591926 -14.488807     11 -15.920032   
275.975000  45    -16.172809  1.709087 -14.463722     11 -16.021059   
299.750000  46    -16.290234  1.824841 -14.465393     11 -16.150294   
323.525000  47    -16.434197  1.957173 -14.477024     11 -16.307530   
347.300000  48    -16.591598  2.125367 -14.466232     11 -16.478191   
371.075000  49    -16.753847  2.327815 -14.426032     11 -16.653269   
394.850000  50    -16.919161  2.556569 -14.362592     11 -16.831062   
418.625000  51    -17.086386  2.807764 -14.278622     11 -17.010221   
442.400000  52    -17.254371  3.078681 -14.175690     11 -17.189331   
466.175000  53    -17.421899  3.366955 -14.054944     11 -17.367467   
489.950000  54    -17.587676  3.670346 -13.917330

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')